In [105]:
#Função que retorna TRUE ou FALSE para contabilizar (ou não) potências negativas

def prop(v1, v2):
    p = 100
    if (v1 == v2):
        p = 0
    if (v1 > v2) and (v1 != 0):
        p = abs(100-((100*v2)/v1))
    if (v2 > v1) and (v2 != 0):
        p = abs(100-((100*v1)/v2))
    return p

def potNegativa(customerType, nElem, vA, vB, vC, avA, avB, avC, iA, iB, iC, aiA, aiB, aiC, fpA, fpB, fpC, pA, pB, pC):
    #customerType pode existir em apenas dois valores
    #  "BT" - para qualquer medição em baixa-tensão
    #  "MT" - para qualquer medição em média-tensão
    #nElem é o numero de elementos de medição (em geral, MT pode ser 3 ou 2, BT (BTI/BT200A) normalmente é 3)
    
    result = False
    if (customerType == "MT"):
        #considera as potências no secundário
        maxPotNegMT = -8 #Watts
        minFPPotNeg = abs(-0.2)
        if ((pA <= maxPotNegMT and abs(fpA) > minFPPotNeg) or (pB <= maxPotNegMT and abs(fpB) > minFPPotNeg)) or ((nElem == 3 and (pC <= maxPotNegMT and abs(fpC) > minFPPotNeg)) or (nElem == 2 and (pC <= maxPotNegMT and abs(fpC) > minFPPotNeg) and (aiC < 30 or aiC > 210))):
            result = True
    if (customerType == "BT"):
        maxPotNegMT = -20 #Watts
        minFPPotNeg = -0.4
        maxPropCorr = 50
        maxAmpere = 0.99
        if (pA <= maxPotNegMT or pB <= maxPotNegMT or pC <= maxPotNegMT):
            #qualquer das tres potencias negativas em principio é para contabilizar
            #só será desarmado se tratar-se de Kirchoff para cargas equilibradas com banco de capacitor e carga não trifasica pequena
            result = True

            if (pA != 0):
                spA = pA/abs(pA)
            else:
                spA = +1
            if (pB != 0):
                spB = pB/abs(pB)
            else:
                spB = +1
            if (pC != 0):
                spC = pC/abs(pC)
            else:
                spC = +1
            if (fpA != 0):
                sfpA = (fpA/abs(fpA)) * spA
            else:
                sfpA = +1
            if (fpB != 0):
                sfpB = (fpB/abs(fpB)) * spB
            else:
                sfpB = +1
            if (fpC != 0):
                sfpC = (fpC/abs(fpC)) * spC
            else:
                sfpC = +1
            if (sfpA == -1 and sfpB == -1 and sfpC == -1):
                #três fatores de potencia sao capacitivos e pequenos
                if ((abs(fpA)*sfpA) > minFPPotNeg and (abs(fpB)*sfpB) > minFPPotNeg and (abs(fpC)*sfpC) > minFPPotNeg):
                    #quantas fases negativas (só pode haver 1)
                    fasesNeg = 0
                    if (spA < 0): 
                        fasesNeg += 1
                    if (spB < 0): 
                        fasesNeg += 1
                    if (spC < 0): 
                        fasesNeg += 1
                    if (fasesNeg == 1):
                        #tres correntes precisam estar relativamente equilibradas
                        propiAB = prop(iA, iB)
                        propiAC = prop(iA, iC)
                        propiBC = prop(iB, iC)
                        if (iA <= maxAmpere and iB <= maxAmpere and iC <= maxAmpere and propiAB < maxPropCorr and propiAC < maxPropCorr and propiBC < maxPropCorr):
                            result = False

    #retorna o resultado
    return result


In [106]:
#MT
res =  potNegativa("MT", 3, 117, 118, 117, 0, 120, 240, 0.01, 0.02, 0.01, 92, 210, 330, 0.2, -0.001, -0.001, -12, 1, 0)
print (res)

False


In [107]:
#BT
res =  potNegativa("BT", 3, 117, 118, 117, 0, 120, 240, 0.76, 0.44, 0.46, 92, 210, 330, 0.1, -0.1, -0.1, -12, 1, 1)
print (res)

False


In [108]:
#teste BT BTI411387957
import numpy as np
import pandas as pd

file = 'fasor_08432289_20190625_110427.csv'
grid=pd.read_csv(file,delimiter=';').astype('object')

In [109]:
grid.head()

,Data,Dia,vA,vB,vC,avA,avB,avC,iA,iB,...,AC,Distorcao Harmonica (%),Frequencia (Hz),Sequencia,1,2,3,1.1,2.1,3.1
0,01/12/2018 00:00,Sabado,135,132,131,0,120,240,0.48,0.46,...,230,0,60,132,44,0,251,12.383.964.0000,547,27.101.060.0000
1,01/12/2018 00:15,Sabado,133,132,132,0,120,240,0.47,0.46,...,229,0,60,132,44,0,250,12.384.008.0000,547,27.101.312.0000
2,01/12/2018 00:30,Sabado,134,131,132,0,120,240,0.47,0.46,...,230,0,60,132,43,0,250,12.384.051.0000,547,27.101.560.0000
3,01/12/2018 00:45,Sabado,132,133,132,0,120,240,0.46,0.44,...,228,0,60,132,46,0,246,12.384.097.0000,547,27.101.808.0000
4,01/12/2018 01:00,Sabado,133,132,133,0,120,240,0.47,0.42,...,230,0,60,132,43,0,249,12.384.140.0000,547,27.102.056.0000


In [110]:
for index, row in grid.iterrows():
    data = row['Data']
    vA = float(row['vA'])
    vB = float(row['vB'])
    vC = float(row['vC'])
    avA = float(row['avA'])
    avB = float(row['avB'])
    avC = float(row['avC'])
    iA = float(row['iA'])
    iB = float(row['iB'])
    iC = float(row['iC'])
    aiA = float(row['aiA'])
    aiB = float(row['aiB'])
    aiC = float(row['aiC'])
    pA = float(row['pA'])
    pB = float(row['pB'])
    pC = float(row['pC'])
    fpA = float(row['fpA'])
    fpB = float(row['fpB'])
    fpC =  float(row['fpC'])

    res = potNegativa("BT", 3, vA, vB, vC, avA, avB, avC, iA, iB, iC, aiA, aiB, aiC, fpA, fpB, fpC, pA, pB, pC)
    if (res):
        print (data)


## Thiago, para cada cliente, preciso dos horarios de funcionamento para dias de semana, sábado e domingo

In [178]:
#estou informando horario 8:00~18:00 para todos os tipos de dias (default)
#para cada cliente, preciso varrer os dados fasoriais para poder montar o resultado

import datetime
from datetime import time
from dateutil import parser

def parseDate(strdate):
    #format ddmmyyyyhhMMss
    try:
        result = parser.parse(strdate[0:2]+"/"+strdate[3:5]+"/"+strdate[6:10]+" "+strdate[11:13]+":"+strdate[14:16]+":00")
    except ValueError:
        #print (strdate)
        result = parser.parse("01/01/1980 00:00:00")
    return result
        
def horarioFuncionamento(hora, dia, semanai=8, semanaf=18, sabadoi=8, sabadof=18, domingoi=8, domingof=18):
    # veja que estou tratando String e tirei as acentuações do arquivo CSV
    # se voce tiver jeito melhor de fazer pode mudar
    if (dia == "Sabado"):
        return (hora >= sabadoi and hora <= sabadof)
    if (dia == "Domingo"):
        return (hora >= domingoi and hora <= domingof)
    return (hora >= semanai and hora <= semanaf)
    
def correnteZero(grid, semanai=8, semanaf=18, sabadoi=8, sabadof=18, domingoi=8, domingof=18):
    #grid é o dataset dos dados fasoriais do mês em questão
    #a função retorna a quantidade de dias onde todas as correntes são zero em uma fase dentro do horário de funcionamento
    #em rdias e o equivalente em minutos (rdias x 15 minutos)
    rdias = 0
    rminutos = 0
    cdata = -1
    for index, row in grid.iterrows():
        datetime = parseDate(row['Data'])
        data = datetime.date()
        hora = datetime.time().hour
        dia = row['Dia']
        iA = float(row['iA'])
        iB = float(row['iB'])
        iC = float(row['iC'])
        if (cdata == -1 or cdata != data):
            if (cdata != -1):
                #print(cdata,"IA",zeroiA,naozeroiA,"IB",zeroiB,naozeroiB,"IC",zeroiC,naozeroiC)
                #assumindo que mensagens possam ser perdidas, o minimo de correntes zeradas em um determinado
                #dia é de 4 horas (16 mensagens)
                if (zeroiA >= 16 and naozeroiA == 0):
                    rdias +=1
                    rminutos +=(rdias*15)
                else:
                    if (zeroiB >= 16 and naozeroiB == 0):
                        rdias +=1
                        rminutos +=(rdias*15)
                    else:
                        if (zeroiC >= 16 and naozeroiC == 0):
                            rdias +=1
                            rminutos +=(rdias*15)
            #novo dia, começa a contar novamente
            cdata = data
            zeroiA, zeroiB, zeroiC = 0,0,0
            naozeroiA, naozeroiB, naozeroiC = 0,0,0
                   
        hf = horarioFuncionamento(hora, dia, semanai, semanaf, sabadoi, sabadof, domingoi, domingof)
        #print(hf,cdata,hora,dia,iA,iB,iC)
        if (hf):
            if (iA == 0):
                zeroiA +=1
            else:
                naozeroiA +=1
            if (iB == 0):
                zeroiB +=1
            else:
                naozeroiB +=1
            if (iC == 0):
                zeroiC +=1
            else:
                naozeroiC +=1

    return rdias, rminutos

#teste
file = 'fasor_09576321_20190702_163802.csv'
grid=pd.read_csv(file,delimiter=';').astype('object')
dias, minutos = correnteZero(grid, 8, 18, 8, 18, 8, 18)
print ("Dias com corrente zero em horario de funcionamento (todas):",dias, "Minutos:", minutos)



Dias com corrente zero em horario de funcionamento (todas): 0 Minutos: 0


In [169]:
grid

,Data,Dia,vA,vB,vC,avA,avB,avC,iA,iB,...,vAC,DH,FQ,SQ,1,2,3,1.1,2.1,3.1
0,01/02/2019 00:00,Sexta,120,121,124,0,120,240,0,0.37,...,211,0,60,132,6,8,0,66.583.00,38.756.00,20
1,01/02/2019 00:15,Sexta,120,121,124,0,120,240,0,0.38,...,211,0,60,132,5,7,0,66.588.00,38.763.00,20
2,01/02/2019 00:30,Sexta,120,123,124,0,120,240,0,0.38,...,211,0,60,132,6,8,0,66.594.00,38.771.00,20
3,01/02/2019 00:45,Sexta,119,121,123,0,120,240,0,0.37,...,209,0,60,132,6,7,0,66.600.00,38.778.00,20
4,01/02/2019 01:00,Sexta,119,120,122,0,120,240,0,0.37,...,208,0,60,132,6,8,0,66.606.00,38.786.00,20
5,01/02/2019 01:15,Sexta,120,122,123,0,120,240,0,0.38,...,210,0,60,132,6,7,0,66.612.00,38.793.00,20
6,01/02/2019 01:30,Sexta,119,121,123,0,120,240,0,0.39,...,209,0,60,132,5,8,0,66.617.00,38.801.00,20
7,01/02/2019 01:45,Sexta,120,122,124,0,120,240,0,0.39,...,211,0,60,132,6,7,0,66.623.00,38.808.00,20
8,01/02/2019 02:00,Sexta,121,123,124,0,120,240,0,0.37,...,212,0,60,132,6,8,0,66.629.00,38.816.00,20
9,01/02/2019 02:15,Sexta,120,123,124,0,120,240,0,0.4,...,211,0,60,132,6,7,0,66.635.00,38.823.00,20


In [2]:
import pandas as pd

In [3]:
#Nova implementação para contabilização de corrente zero
#Novidades:
# a) Lista de feriados
# b) Regra de contabilização nao se restringe à corrente ser zero apenas dentro do horário de funcionamento e do feriado

import datetime
from datetime import datetime
from datetime import time
from dateutil import parser

def parseDate(strdate):
    #format ddmmyyyyhhMMss
    try:
        result = parser.parse(strdate[0:2]+"/"+strdate[3:5]+"/"+strdate[6:10]+" "+strdate[11:13]+":"+strdate[14:16]+":00", dayfirst=True)
    except ValueError:
        #print (strdate)
        result = parser.parse("01/01/1980 00:00:00", dayfirst=True)
    return result
        
def horarioFuncionamento(hora, dia, semanai=8, semanaf=18, sabadoi=8, sabadof=18, domingoi=8, domingof=18):
    # veja que estou tratando String e tirei as acentuações do arquivo CSV
    # se voce tiver jeito melhor de fazer pode mudar
    if (dia == "Sabado"):
        horas = sabadof - sabadoi
        return (hora >= sabadoi and hora <= sabadof), horas
    if (dia == "Domingo"):
        horas = domingof - domingoi
        return (hora >= domingoi and hora <= domingof), horas
    else:
        horas = semanaf - semanai
        return (hora >= semanai and hora <= semanaf), horas
    
def verificaFeriado(datastr, feriados):
    return (feriados['Feriados'] == datastr).any()
    
def NEWcorrenteZero(grid, feriados, pAmp, relacao, semanai=8, semanaf=18, sabadoi=8, sabadof=18, domingoi=8, domingof=18):
    rdias = 0
    rminutos = 0
    severidade_regra = 0 #certeza de que nao tem nada errado
    cdata = -1
    
    corrente_referencia = pAmp/relacao
    iA_levanta = (grid['iA'] >= corrente_referencia/3).any()
    iB_levanta = (grid['iB'] >= corrente_referencia/3).any()
    iC_levanta = (grid['iC'] >= corrente_referencia/3).any()

    print("Corrente de referencia:", corrente_referencia)
    print("iA tem valores maiores que referencia/3: ", iA_levanta)
    print("iB tem valores maiores que referencia/3: ", iB_levanta)
    print("iC tem valores maiores que referencia/3: ", iC_levanta)

    for index, row in grid.iterrows():
        datetime = parseDate(row['Data'])
        data = datetime.date()
        datastr = datetime.strftime("%d/%m/%Y")
        hora = datetime.time().hour
        dia = row['Dia']
        iA = float(row['iA'])*relacao
        iB = float(row['iB'])*relacao
        iC = float(row['iC'])*relacao
        if (cdata == -1 or cdata != data):
            feriado = verificaFeriado(datastr, feriados)
            #print("data: ", datastr, feriado)
            if (cdata != -1):
                ocorrencias = (horas*4)*0.7 #70%do tempo
                print(cdata,"Ref Ocorrencias:",ocorrencias,"IA",zeroiA,naozeroiA,"IB",zeroiB,naozeroiB,"IC",zeroiC,naozeroiC)
                if (zeroiA >= ocorrencias): # and naozeroiA == 0):
                    rdias +=1
                    rminutos +=(zeroiA*15)
                else:
                    if (zeroiB >= ocorrencias): # and naozeroiB == 0):
                        rdias +=1
                        rminutos +=(zeroiB*15)
                    else:
                        if (zeroiC >= ocorrencias): # and naozeroiC == 0):
                            rdias +=1
                            rminutos +=(ZeroiC*15)
            #novo dia, começa a contar novamente
            cdata = data
            zeroiA, zeroiB, zeroiC = 0,0,0
            naozeroiA, naozeroiB, naozeroiC = 0,0,0
                   
        hf,horas = horarioFuncionamento(hora, dia, semanai, semanaf, sabadoi, sabadof, domingoi, domingof)
        #print(hf,cdata,hora,dia,iA,iB,iC)
        
        if (hf and not feriado):
            if (iA == 0):
                severidade_regra = 1
                if (iA+iB+iC >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeroiA +=1
                else:
                    if (not iA_levanta):
                        #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                        zeroiA +=1
            else:
                naozeroiA +=1
            if (iB == 0):
                severidade_regra = 1
                if (iA+iB+iC >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeroiB +=1
                else:
                    if (not iB_levanta):
                        #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                        zeroiB +=1
            else:
                naozeroiB +=1
            if (iC == 0):
                severidade_regra = 1
                if (iA+iB+iC >= pAmp):
                    #condição C = corrente zero e a soma das outras duas é maior ou igual a pAmp
                    zeroiC +=1
                else:
                    if (not iC_levanta):
                        #condição A = corrente zero na fase A sendo que a fase A nunca ultrapassa 1/3 de pAmp
                        zeroiC +=1
            else:
                naozeroiC +=1

    #so para ajustar severidade_regra
    if (rdias >= 10): # nao importa rminutos
        severidade_regra = 2
        
    return rdias, rminutos, severidade_regra

#teste
file = 'BTI410315485-fasor_08552562_20190711_153409.csv'
grid=pd.read_csv(file,delimiter=';').astype('object')
file = 'feriados.csv'
feriados=pd.read_csv(file,delimiter=';').astype('object')
pAmp = 20
relacao = 400/5 # Observacao: cliente BT2A a relação é 1/1

dias, minutos, severidade_regra = NEWcorrenteZero(grid, feriados, pAmp, relacao, 8, 18, 8, 18, 8, 18)
print ("Dias com corrente zero em horario de funcionamento:",dias, " --- Minutos:", minutos, " (", severidade_regra, ")")

Corrente de referencia: 0.25
iA tem valores maiores que referencia/3:  True
iB tem valores maiores que referencia/3:  True
iC tem valores maiores que referencia/3:  True
2019-02-27 Ref Ocorrencias: 28.0 IA 0 44 IB 0 44 IC 0 44
2019-02-28 Ref Ocorrencias: 28.0 IA 0 44 IB 0 44 IC 0 44
2019-03-01 Ref Ocorrencias: 28.0 IA 0 44 IB 0 41 IC 0 44
2019-03-02 Ref Ocorrencias: 28.0 IA 0 0 IB 0 0 IC 0 0
2019-03-03 Ref Ocorrencias: 28.0 IA 0 0 IB 0 0 IC 0 0
2019-03-04 Ref Ocorrencias: 28.0 IA 0 0 IB 0 0 IC 0 0
2019-03-05 Ref Ocorrencias: 28.0 IA 0 0 IB 0 0 IC 0 0
2019-03-06 Ref Ocorrencias: 28.0 IA 0 0 IB 0 0 IC 0 0
2019-03-07 Ref Ocorrencias: 28.0 IA 0 44 IB 0 44 IC 0 44
2019-03-08 Ref Ocorrencias: 28.0 IA 0 44 IB 0 44 IC 0 44
2019-03-09 Ref Ocorrencias: 28.0 IA 0 44 IB 0 33 IC 0 44
2019-03-10 Ref Ocorrencias: 28.0 IA 0 44 IB 0 0 IC 0 44
2019-03-11 Ref Ocorrencias: 28.0 IA 0 44 IB 0 44 IC 0 44
2019-03-12 Ref Ocorrencias: 28.0 IA 0 44 IB 0 44 IC 0 44
2019-03-13 Ref Ocorrencias: 28.0 IA 0 43 IB 0 43